<a href="https://colab.research.google.com/github/akuzikemoyo/AI-class/blob/main/CHATBOT_RAG_EXERCISE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-openai -q
!pip install langchain-community -q
!pip install langchain-experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.1 MB/s eta 0:00:00


In [ ]:

from google.colab import userdata
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from typing import TypedDict, List, Optional, Tuple, Dict
import gradio as gr

In [ ]:

from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda
import time

In [ ]:
class ZhipuAI_embeddings:
    def __init__(self, model_name: str = 'embeddings-3'):
        self.model_name = model_name
        self.base_url = "https://open.bigmodel.cn/api/paas/v4"
        self.embedding = self._init_model()
    def _init_model(self) -> OpenAIEmbeddings:
        return OpenAIEmbeddings(
            model=self.model_name,
            base_url=self.base_url,
            api_key=userdata.get("GLYN")
        )

embeddings = ZhipuAI_embeddings().embedding


In [ ]:
client = ChatOpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4/",
    api_key=userdata.get("GLYN"),
    model="glm-4.5"
)


In [ ]:
def doc_parsing(file_path) -> list[Document]:
    if file_path.endswith("pdf"):
        loader = PyPDFLoader(file_path=file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path=file_path)
    else:
        return []

    try:
        doc = loader.load()
    except Exception as e:
        print(f"Error loading document: {e}")
        return []

    semantic_splitter = SemanticChunker(
        embeddings=embeddings,
        breakpoint_threshold_type="percentile",
        breakpoint_threshold_amount=95
    )
    full_text = doc[0].page_content if doc else ""
    if not full_text:
        return []

    raw_chunks = semantic_splitter.split_text(full_text)
    print(f"Number of chunks created: {len(raw_chunks)}")
    docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
    return docs

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
@tool
def define_term(term: str) -> str:
    """Returns a one-sentence definition of a given AI term. Use this tool when a student asks 'What is X?' or 'define Y'."""
    definitions = {
        "transformer": "A model architecture that uses attention mechanisms to weigh the importance of different parts of the input sequence.",
        "embedding": "A numerical representation of text that captures its semantic meaning for use in machine learning models.",
        "rag": "A technique that retrieves information from an external knowledge base to improve the accuracy and relevance of an LLM's response.",
    }
    return definitions.get(term.lower(), f"I'm sorry, I don't have a definition for '{term}'.")


In [ ]:
@tool
def summarize_notes(text: str) -> str:
    """Takes a block of student notes (a paragraph) and returns a concise summary of 2-3 sentences. Use this tool when a student provides a long paragraph and asks to summarize it."""
    summary_chain = ChatPromptTemplate.from_template(
        "Summarize the following text in 2-3 sentences:\n\n{text}"
    ) | client
    return summary_chain.invoke({"text": text}).content


In [ ]:
def retrieve_docs(question: str) -> str:
    """Retrieves relevant documents from the vector store based on a student's question."""
    if not vector_store.get_all_documents():
        return "No documents have been uploaded to search from."

    SUB_QUERY_TEMPLATE = """
    You are a helpful assistant that generates multiple search queries based on a single input query.
    Generate 3 diverse search queries related to the user's question, which can be used to retrieve relevant documents.
    The queries should be concise and cover different aspects or angles of the original question.

    Original Question: {question}

    Generated Queries:
    -
    """
    sub_query_chain = ChatPromptTemplate.from_template(SUB_QUERY_TEMPLATE) | client
    response = sub_query_chain.invoke({"question": question})
    queries = [q.strip() for q in response.content.split('-') if q.strip()]

    all_retrieved_docs = []
    seen_doc_contents = set()
    for query in queries:
        retrieved_for_query = vector_store.similarity_search(query)
        for doc in retrieved_for_query:
            if doc.page_content not in seen_doc_contents:
                all_retrieved_docs.append(doc)
                seen_doc_contents.add(doc.page_content)

    if not all_retrieved_docs:
        return "No relevant information found in the uploaded documents."

    context_text = "\n\n".join(d.page_content for d in all_retrieved_docs)
    return context_text

In [ ]:
rag_tool = tool(retrieve_docs)

# List available tools for the agent
tools = [define_term, summarize_notes, rag_tool]

# Create the agent with its prompt
template_with_tools = """
You are AICLASS assistant, a smart and helpful assistant at takenolab. You have access to specialized tools to help students with their course content.
Your primary goal is to be supportive, precise, and direct.

You must answer student questions by first determining if you need a tool.
- If the question is about an AI term, use the `define_term` tool.
- If the request is to summarize text, use the `summarize_notes` tool.
- For all other questions about course content (e.g., "What does the policy say about...?", "Explain topic X"), use the `retrieve_course_content` tool.
- If a question is not related to any of these tasks, kindly state that you cannot assist.

student problem:
{input}

{agent_scratchpad}
"""

prompt = ChatPromptTemplate.from_template(template_with_tools)
llm_with_tools = client.bind_tools(tools)
agent = create_tool_calling_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
def retrieve_and_answer_with_history(question: str, chat_history: List[Dict[str, str]]) -> List[Dict[str, str]]:
    # Convert chat_history (list of dictionaries) to list of BaseMessage for Langchain
    history_messages: List[BaseMessage] = []
    for message in chat_history:
        if message["role"] == "user":
            history_messages.append(HumanMessage(content=message["content"]))
        elif message["role"] == "assistant":
            history_messages.append(AIMessage(content=message["content"]))

    try:
        # Invoke the agent with the current question and formatted history
        response = agent_executor.invoke({"input": question, "chat_history": history_messages})
        bot_message = response.get("output", "I'm sorry, I couldn't generate a response.")

        # Append the new interaction to the history in the correct format
        chat_history.append({"role": "user", "content": question})
        chat_history.append({"role": "assistant", "content": bot_message})

        return chat_history

    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        chat_history.append({"role": "user", "content": question})
        chat_history.append({"role": "assistant", "content": error_message})
        return chat_history

In [ ]:
def doc_loader(file_path):
    docs = doc_parsing(file_path)
    if not docs:
        return "No content found or processed in the document."
    _ = vector_store.add_documents(documents=docs)
    return f"Successfully added {len(docs)} document chunks."

In [ ]:
def interface():
    iface = gr.ChatInterface(
        fn=retrieve_and_answer_with_history,
        chatbot=gr.Chatbot(height=200, type='messages', label="Assistant"),
        textbox=gr.Textbox(lines=2,submit_btn=True ),
        title="Takenolab AIClass Assistant (Conversational RAG)",
        type='messages',
        description="Ask a question about your course content and get smart advice, supporting multi-turn conversations.",
    )
    docs_interface = gr.Interface(
        fn=doc_loader,
        inputs=gr.File(label="Choose a file to upload",
                       type='filepath',
                       file_count='single',
                       show_label=True
                       ),
        description="Upload a document to run retrieval‐augmented generation.",
        outputs=gr.TextArea()
    )
    table = gr.TabbedInterface(
        [iface,docs_interface],
        tab_names= ['Chat', "Upload File for RAG"],
        title="LLM, RAG AND PROMPTS, Text Generation"
    )
    iface.launch(debug=True, server_port=3000)


In [ ]:
interface()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://974b0cc006747a2fc4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...


Hello! I'm doing well, thank you for asking. I'm here to help you with your AI course content. If you have any questions about AI terms, need help summarizing notes, or want to explore course topics, feel free to ask!

> Finished chain.
